In [11]:
import pandas as pd
from bs4 import BeautifulSoup

In [6]:
# Import Stackoverflow data https://www.kaggle.com/datasets/stackoverflow/stacksample?resource=download
questions = pd.read_csv("./data/Questions.csv", encoding="ISO-8859-1")
answers = pd.read_csv("./data/Answers.csv", encoding="ISO-8859-1")

In [13]:
print(f"questions shape: {questions.shape}")
print(f"questions columns: {questions.columns}")

questions shape: (1264216, 7)
questions columns: Index(['Id', 'OwnerUserId', 'CreationDate', 'ClosedDate', 'Score', 'Title',
       'Body'],
      dtype='object')


In [14]:
print(f"answers shape: {answers.shape}")
print(f"answers columns: {answers.columns}")

answers shape: (2014516, 6)
answers columns: Index(['Id', 'OwnerUserId', 'CreationDate', 'ParentId', 'Score', 'Body'], dtype='object')


## Text processing

In [7]:
questions.head()

,Id,OwnerUserId,CreationDate,ClosedDate,Score,Title,Body
0,80,26.0,2008-08-01T13:57:07Z,NaN,26,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...
1,90,58.0,2008-08-01T14:41:24Z,2012-12-26T03:45:49Z,144,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...
2,120,83.0,2008-08-01T15:50:08Z,NaN,21,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...
3,180,2089740.0,2008-08-01T18:42:19Z,NaN,53,Function for creating color wheels,<p>This is something I've pseudo-solved many t...
4,260,91.0,2008-08-01T23:22:08Z,NaN,49,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...


In [8]:
answers.head()

,Id,OwnerUserId,CreationDate,ParentId,Score,Body
0,92,61.0,2008-08-01T14:45:37Z,90,13,"<p><a href=""http://svnbook.red-bean.com/"">Vers..."
1,124,26.0,2008-08-01T16:09:47Z,80,12,<p>I wound up using this. It is a kind of a ha...
2,199,50.0,2008-08-01T19:36:46Z,180,1,<p>I've read somewhere the human eye can't dis...
3,269,91.0,2008-08-01T23:49:57Z,260,4,"<p>Yes, I thought about that, but I soon figur..."
4,307,49.0,2008-08-02T01:49:46Z,260,28,"<p><a href=""http://www.codeproject.com/Article..."


In [15]:
# select relevant columns
questions = questions[["Id", "Body"]]
answers = answers[["ParentId", "Body"]]

In [20]:
q_and_a = answers.merge(questions, left_on="ParentId", right_on="Id", suffixes=('_a', '_q'), how="left")[["Body_a", "Body_q"]]

In [22]:
q_and_a.head()

,Body_a,Body_q
0,"<p><a href=""http://svnbook.red-bean.com/"">Vers...",<p>Are there any really good tutorials explain...
1,<p>I wound up using this. It is a kind of a ha...,<p>I've written a database generation script i...
2,<p>I've read somewhere the human eye can't dis...,<p>This is something I've pseudo-solved many t...
3,"<p>Yes, I thought about that, but I soon figur...",<p>I have a little game written in C#. It uses...
4,"<p><a href=""http://www.codeproject.com/Article...",<p>I have a little game written in C#. It uses...


In [24]:
q_and_a['Body_a'] = q_and_a['Body_a'].apply(lambda x: BeautifulSoup(x).get_text())
q_and_a['Body_q'] = q_and_a['Body_q'].apply(lambda x: BeautifulSoup(x).get_text()) 

In [25]:
q_and_a.head()

,Body_a,Body_q
0,Version Control with Subversion\nA very good r...,Are there any really good tutorials explaining...
1,"I wound up using this. It is a kind of a hack,...",I've written a database generation script in S...
2,I've read somewhere the human eye can't distin...,This is something I've pseudo-solved many time...
3,"Yes, I thought about that, but I soon figured ...",I have a little game written in C#. It uses a ...
4,Oleg Shilo's C# Script solution (at The Code P...,I have a little game written in C#. It uses a ...


In [26]:
q_and_a.rename(columns={"Body_q": "questions", "Body_a": "answers"}, inplace=True)

In [27]:
q_and_a.to_csv("./data/Questions_And_Answers.csv")